# Case-study: Cross-design impact

In [1]:
# Get python executable for jupyter notebook
import sys
print(sys.executable)

/Library/Developer/CommandLineTools/usr/bin/python3


In [2]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

optional arguments:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [5]:
!rm -f ref_output/new_arch.yaml ref_output/new_arch.yaml.*
!safinfer -a ref_input/arch.yaml -s ref_input/sparseopts_B_bidir.yaml -q ref_input/safinfer_settings_B_bidir.yaml -o ref_output/ -L
!mv ref_output/new_arch.yaml.pretty ref_output/new_arch_B_bidir.yaml.pretty
!mv ref_output/new_arch.yaml ref_output/new_arch_B_bidir.yaml
!mv safinfer.log safinfer_B_bidir.log

- SAFinfer settings path: ref_input/safinfer_settings_B_bidir.yaml
logging: True
>> SAFinfer
:: Setup
:: => Done, setup
reconfigurable_arch: False
:: Taxonomic inference
- COMPLETE: microarchitecture inference
  => SUCCESS
- Dumping inferred SAF microarchitecture topology to ref_output/new_arch.yaml ...
:: => Done, taxonomic inference
<< Done, SAFinfer


In [43]:
!safmodel -h

usage: safmodel [-h] [-a ARCH] [-b CHAR] [-c COMP_IN] [-d RTL] [-f LOG_FILE]
                [-i DIR_IN] [-j HCL] [-k COMP_OUT] [-L] [-m MODEL_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-o DIR_OUT] [-r ARCH_OUT] [-s SPARSEOPTS]
                [-t TAXO_SCRIPT_LIB] [-U SETTINGS] [-u SIM]

safmodel: SAF microarchitecture modeling generator.

options:
  -h, --help            show this help message and exit
  -a ARCH, --arch ARCH  Sparseloop architecture file.
  -b CHAR, --char CHAR  CSV EAT characterization table.
  -c COMP_IN, --comp-in COMP_IN
                        One or more Accelergy component library YAML
                        filenames.
  -d RTL, --rtl RTL     Path to underlying RTL files associated with CSV EAT
                        characterization table (-b/--char)
  -f LOG_FILE, --log-file LOG_FILE
                        Log filepath (requires -L/--log to enable logging).
  -i DIR_IN, --dir-in DIR_IN
                        Input files' directory. If not specified, a

# Fake SAFModel

In [1]:
import safmodel
import core.helper as helper
from core.helper import info,warn,error
from core import safmodel_core as safcore, \
                 safmodel_io as safio, \
                 safinfer_io
import os

In [2]:
# Spoof CLI flag parsing

import sys

sys.argv = [
    'safmodel', # This is typically the script name
    '-T', 'ref_output/new_arch_B_bidir.yaml',
    '-a', 'ref_input/arch.yaml',
    '-s', 'ref_input/sparseopts_B_bidir.yaml',
    '-c', 'ref_input/compound_components.yaml',
    '-U', 'ref_input/safmodel_settings.yaml',
    '-r', 'ref_output/arch_w_SAF.yaml',
    '-k', 'ref_output/',
    '-L'
]

arch, \
taxo_uarch, \
sparseopts, \
comp_in, \
arch_out_path, \
comp_out_path, \
user_attributes, \
do_logging,\
log_fn, \
characterization_path_list, \
model_script_lib_list, \
taxo_script_lib_list = safio.parse_args()

safmodel.log_config(do_logging,log_fn)
safmodel.opening_remark()
safmodel.setup(taxo_script_lib_list,characterization_path_list,model_script_lib_list)

SAFmodel.

Parsing input files:
- netlist: ref_output/new_arch_B_bidir.yaml
- arch: ref_input/arch.yaml
- sparseopts: ref_input/sparseopts_B_bidir.yaml
- compound components (input): ['ref_input/compound_components.yaml']
- arch output path: ref_output/arch_w_SAF.yaml
- compound components path (output): ref_output/
- SAFModel settings path: ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup


In [3]:
# Build scale inference problem

warn(":: Scale inference",also_stdout=True)
'''Build scale inference problem'''
scale_prob=safcore.build_scale_inference_problem(arch, sparseopts, taxo_uarch, user_attributes=user_attributes)

:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 100%|██████████| 7/7 [00:00<00:00, 146.81it/s]
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-25
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 100%|██████████| 7/7 [00:00<00:00, 172.15it/s]
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-

In [4]:
list(scale_prob.keys())

['symbols',
 'symbol_types',
 'constraints',
 'energy_objectives',
 'area_objectives',
 'yields',
 'primitive_models',
 'component_models',
 'buffer_action_tree',
 'component_energy_action_tree',
 'sub_action_graph',
 'global_objective',
 'global_energy_objective',
 'global_area_objective',
 'abstract_global_objective_expression',
 'area_multiplier_dict',
 'simplified_symbols',
 'simplified_symbol_types',
 'simplified_constraints',
 'user_attributes']

In [6]:
scale_prob['global_objective']

'(-1.440167358764436e-6*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_rw_thresh**3 + 0.000227214652926367*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_rw_thresh**2*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_ww_thresh + 3.229989646838836e-5*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_rw_thresh**2 - 0.000141976287723061*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_rw_thresh*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_ww_thresh**2 + 0.01298394878636807*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_rw_thresh*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_ww_thresh - 0.001068796000228397*TestArchitecture.Skipping_shared_buffer3_shared_buffer1.DoublePgen.md_in0_ref_rw_thresh + 0.0001432443977500827*TestArchitecture.Skipping_shared_b

In [8]:
scale_prob_filename = 'scale_prob.dill'
simplified_constraints_filename = 'simplified_constraints.txt'
simplified_symbols_filename = 'simplified_symbols.txt'
simplified_symbol_types_filename = 'simplified_symbol_types.txt'
global_objective_filename = 'global_objective_file.yaml'
#area_objectives_filename = 'area_objectives_file.yaml'

# Dumping scale problem structure

import dill
# Open the file in binary write mode and save the variable
with open(scale_prob_filename, 'wb') as file:
    dill.dump(scale_prob, file)

print(f"Variable 'scale_prob' has been saved to {scale_prob_filename}")

# Dumping 

# Assuming scale_prob is your dictionary and 'simplified_constraints' is the key
simplified_constraints = scale_prob['simplified_constraints']
simplified_symbols = scale_prob['simplified_symbols']
simplified_symbol_types = scale_prob['simplified_symbol_types']
global_objective = scale_prob['global_objective']
#area_objectives = scale_prob['area_objectives']

# Specify the filename for the text file


# Constraints
with open(simplified_constraints_filename, 'w') as file:
    for constraint in simplified_constraints:
        file.write(constraint + '\n')

# Symbols
with open(simplified_symbols_filename, 'w') as file:
    for symbol in simplified_symbols:
        file.write(symbol + '\n')

# Symbol types
with open(simplified_symbol_types_filename, 'w') as file:
    for symbol_type in simplified_symbol_types:
        file.write(symbol_type + '\n')

import yaml
# Energy objectives dump
with open(global_objective_filename, 'w') as file:
    yaml.dump(global_objective, file)


#print(f"List 'simplified_constraints' has been saved to {simplified_symbol_types_filename}")


Variable 'scale_prob' has been saved to scale_prob.dill


## Reload scale problem & modified constraints

In [26]:
modified_symbols_filename = "modified_symbols.txt"
modified_symbol_types_filename = "modified_symbol_types.txt"
modified_constraints_filename = "modified_constraints.txt"
modified_global_objective_filename = 'modified_global_objective_file.yaml'
#modified_area_objectives_filename = 'modified_area_objectives_file.yaml'

# Reload scale problem
with open(scale_prob_filename, 'rb') as file:
    modified_scale_prob = dill.load(file)

# Load modified constraints
with open(modified_constraints_filename, 'r') as file:
    modified_constraints = [line.strip() for line in file]

# Load modified constraints
with open(modified_symbols_filename, 'r') as file:
    modified_symbols = [line.strip() for line in file]

# Load modified constraints
with open(modified_symbol_types_filename, 'r') as file:
    modified_symbol_types = [line.strip() for line in file]

with open(modified_global_objective_filename, 'r') as file:
    modified_global_objective = yaml.safe_load(file)

# Overwrite symbols
modified_scale_prob['simplified_symbols'] = modified_symbols

# Overwrite symbol tyeps
modified_scale_prob['simplified_symbol_types'] = modified_symbol_types

# Overwrite constraints list in scale problem
modified_scale_prob['simplified_constraints'] = modified_constraints

# Overwrite energy objectives
modified_scale_prob['global_objective'] = modified_global_objective

In [27]:
# Solve scale inference problem
abstract_analytical_primitive_models_dict,abstract_analytical_component_models_dict= \
    safcore.solve_scale_inference_problem(modified_scale_prob)

Solving scale inference problem.
Constraint constr_0[None]: TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_rw  <=  TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_rw_thresh
Constraint constr_1[None]: TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_pr  <=  TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_pr_thresh
Constraint constr_2[None]: TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_ww  <=  TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_ww_thresh
Constraint constr_3[None]: TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.at_bound_out_pr*TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_nc  ==  TestArchitecture.shared_buffer_datatype_format_uarch.TestMetadataParser0.md_in_cr
Constraint constr_4[None]: TestArchitecture.shared_buffer_datatype_format

ActionManagerError: Problem executing an event.  No results are available.

## End

In [11]:
!rm ref_output/arch_w_SAF ref_output/arch_w_SAF_C_bidir.yaml ref_output/bindings_C_bidir.yaml
!safmodel -T ref_output/new_arch_C_bidir.yaml -a ref_input/arch.yaml -s ref_input/sparseopts_C_bidir.yaml -c ref_input/compound_components.yaml -U ref_input/safmodel_settings.yaml  -r ref_output/arch_w_SAF.yaml -k   ref_output/ -L
!mv ref_output/arch_w_SAF.yaml ref_output/arch_w_SAF_C_bidir.yaml
!mv safmodel.log safmodel_C_bidir.log

rm: ref_output/arch_w_SAF: No such file or directory
rm: ref_output/arch_w_SAF_C_bidir.yaml: No such file or directory


SAFmodel.

Parsing input files:
- netlist: ref_output/new_arch_C_bidir.yaml
- arch: ref_input/arch.yaml
- sparseopts: ref_input/sparseopts_C_bidir.yaml
- compound components (input): ['ref_input/compound_components.yaml']
- arch output path: ref_output/arch_w_SAF.yaml
- compound components path (output): ref_output/
- SAFModel settings path: ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-25
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$(u) energy ar

In [12]:
!rm ref_output/arch_w_SAF ref_output/arch_w_SAF_C_lf.yaml ref_output/bindings_C_lf.yaml
!safmodel -T ref_output/new_arch_C_lf.yaml -a ref_input/arch.yaml -s ref_input/sparseopts_C_lf.yaml -c ref_input/compound_components.yaml -U ref_input/safmodel_settings.yaml  -r ref_output/arch_w_SAF.yaml -k   ref_output/ -L
!mv ref_output/arch_w_SAF.yaml ref_output/arch_w_SAF_C_lf.yaml
!mv safmodel.log safmodel_C_lf.log

rm: ref_output/arch_w_SAF: No such file or directory
rm: ref_output/arch_w_SAF_C_lf.yaml: No such file or directory
rm: ref_output/bindings_C_lf.yaml: No such file or directory


SAFmodel.

Parsing input files:
- netlist: ref_output/new_arch_C_lf.yaml
- arch: ref_input/arch.yaml
- sparseopts: ref_input/sparseopts_C_lf.yaml
- compound components (input): ['ref_input/compound_components.yaml']
- arch output path: ref_output/arch_w_SAF.yaml
- compound components path (output): ref_output/
- SAFModel settings path: ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.525611648803889e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 9.707226444223633e-25
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$(u) energy area lat